In [9]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.models import load_model

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error

import numpy as np
import pandas as pd

2023-07-06 11:51:05.569001: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-06 11:51:05.570344: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-06 11:51:05.596271: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-06 11:51:05.596858: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-06 11:51:06.226982: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

In [2]:
dados = pd.read_excel('../codigos/data/2019.xlsx')
ativos = dados.columns[1:]

In [3]:
ativos

Index(['ABEV3', 'B3SA3', 'BBAS3', 'BBDC3', 'BBDC4', 'BEEF3', 'BRAP4', 'BRFS3',
       'BRKM5', 'CCRO3', 'CIEL3', 'CMIG4', 'CPFE3', 'CPLE6', 'CSAN3', 'CSNA3',
       'CYRE3', 'EGIE3', 'ELET3', 'ELET6', 'EMBR3', 'ENBR3', 'ENEV3', 'ENGI11',
       'EQTL3', 'EZTC3', 'FLRY3', 'GGBR4', 'GOAU4', 'GOLL4', 'HYPE3', 'ITSA4',
       'ITUB4', 'JBSS3', 'JHSF3', 'LREN3', 'MRFG3', 'MRVE3', 'MULT3', 'PETR3',
       'PETR4', 'RADL3', 'RENT3', 'SANB11', 'SBSP3', 'TAEE11', 'TIMS3',
       'TOTS3', 'UGPA3', 'USIM5', 'VALE3', 'VIVT3', 'WEGE3', 'YDUQ3'],
      dtype='object')

In [5]:
d = pd.DataFrame(data={"ativos":ativos})
d.to_excel('resultados.xlsx', index=False)

In [13]:
from scipy import stats
from sklearn.discriminant_analysis import StandardScaler


def janelaDeslizante(vetor, janelaEntrada, janelaSaida):
    """
    Calcula a matriz de conhecimento
    Args:
        vetor: Array com os valores da série temporal.
        janelaEntrada: int com o tamanho da janela de entrada.
        JanelaSaida: int com o tamanho da janela de saída.

    Returns:
        X: matriz com as entradas do modelo tem tamanho (numero de amostras X tamanho da janela entrada)
        y: matriz com as saídas do modelo tem tamanho (numero de amostras X tamanho da janela de saida)
    """
    #### YEO johnson
    transformacao, lambda_param = stats.yeojohnson(vetor)
    X_y = transformacao
    
    ## Padronização
    scaler = StandardScaler()
    f = np.array(X_y).reshape(-1,1)
    X_p = scaler.fit_transform(f)
    
    X_p = np.array(X_p).flatten()
    
    num_amostras = len(X_p) - janelaEntrada - janelaSaida + 1
    X = np.zeros((num_amostras, janelaEntrada))
    y = np.zeros((num_amostras, janelaSaida))
    for i in range(num_amostras):
        X[i] = X_p[i : i + janelaEntrada]
        y[i] = X_p[i + janelaEntrada : i + janelaEntrada + janelaSaida]

    return X, y

In [12]:
def dividirConjuntoDados(X,y, tamanhoTeste=0.2):
  """
    Divide o conjunto de dados em treino e teste dado a porcentagem para o teste, sem realizar o embaralhamento
    
    Args:
        X: matriz com as entradas do modelo tem tamanho (numero de amostras X tamanho da janela entrada)
        Y: matriz com as saídas do modelo tem tamanho (numero de amostras X tamanho da janela de saida)
        tamanhoTeste: float porcentagem referente ao tamanho do teste

    Returns:
        y_pred: matriz com as previsoes realizadas pelo modelo ajustado
    """
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=tamanhoTeste, random_state=42, shuffle=False)
  
  return X_train,y_train,X_test,y_test

In [ ]:
################################################################
### RESULTADOS INDIVIDUAIS


train = []
test = []
## MODELO INDIVIDUAL
dados = pd.read_excel('../codigos/data/2019.xlsx')
ativos = dados.columns[1:]

for ativo in ativos:
# ativo = 'ABEV3'
  X,y = janelaDeslizante(dados[ativo], 22,1)
  X_train,y_train, X_test, y_test = dividirConjuntoDados(X,y)
  modelo = tf.keras.models.load_model(f'models/individuais/{ativo}.h5')
  y_pred_train = modelo.predict(X_train)
  train.append(mean_absolute_percentage_error(y_train, y_pred_train))
  y_pred_test = modelo.predict(X_test)
  test.append(mean_absolute_percentage_error(y_test, y_pred_test))

In [ ]:
resultado = pd.read_excel('resultados.xlsx')
resultado["Mdelo Individual Treino"] = train
resultado["Mdelo Individual Teste"] = test
resultado.to_excel('resultado.xlsx', index=False)

In [24]:
## MODELO POR GRUPOS
classificacao = pd.read_excel('../codigos/resultados/2019/classes.xlsx')
dados = pd.read_excel('../codigos/data/2019.xlsx')

classes = [0,1,2,3,4,5,6,7]

for classe in classes:

    filtro = classificacao.query(f'classes=={classe}')
    ativos = filtro['ativos']


    ## CONTROI A MATRIZ DE CONHECIMENTO PARA OS ATIVOS DO CLUSTER
    for i,ativo in enumerate(ativos):
    if i == 0 :
        X,y = janelaDeslizante(dados[ativo], 22,1)
    else:
        X_temp,y_temp = janelaDeslizante(dados[ativo], 22,1)
        X = np.concatenate((X,X_temp), axis=0)
        y = np.concatenate((y,y_temp), axis=0)
    print(ativo, i)

    ## MODELAGEM
    X_train,y_train, X_test, y_test = dividirConjuntoDados(X,y)


    modelo = tf.keras.models.load_model(f'models/agrupados/modeloAgrupadoClasse_{classe}.h5')
    y_pred_train = modelo.predict(X_train)
    train.append(mean_absolute_percentage_error(y_train, y_pred_train))
    y_pred_test = modelo.predict(X_test)
    test.append(mean_absolute_percentage_error(y_test, y_pred_test))

In [25]:
### MODELO COM A MEDIA
classificacao = pd.read_excel('../codigos/resultados/2019/classes.xlsx')
dados = pd.read_excel('../codigos/data/2019.xlsx')
classes = [0,1,2,3,4,5,6,7]

for classe in classes:
  filtro = classificacao.query(f'classes=={classe}')
  ativos = filtro['ativos']

  dados['media'] = dados[ativos].T.mean()

  X,y = janelaDeslizante(dados['media'], 22,1)

  X_train,y_train, X_test, y_test = dividirConjuntoDados(X,y)

  modelo = tf.keras.models.load_model(f'models/media/classe_{classe}.h5')
  y_pred_train = modelo.predict(X_train)
  train.append(mean_absolute_percentage_error(y_train, y_pred_train))
  y_pred_test = modelo.predict(X_test)
  test.append(mean_absolute_percentage_error(y_test, y_pred_test))

0.5792208177681912

In [26]:
resultado = pd.read_excel('resultados.xlsx')
resultado["Mdelo com a Média Treino"] = train
resultado["Mdelo com a Média Teste"] = test
resultado.to_excel('resultado.xlsx', index=False)

0.5792208177681912